# Part I: Data Extraction

In [11]:
import praw
import pandas as pd

In [12]:
def data_fetch(sub_red_lst = "Jobs", limit = 1000, time_filter = "all"):
    reddit = praw.Reddit(client_id="HB-6S3t-mU0H4JnFryAcAA", 
                         client_secret="iOPDBfwO35IWt4Wd3lbPTbKbZlS0DQ",
                         redirect_uri="http://localhost:8080", 
                         user_agent="Poliscimania")
    
    data = reddit.subreddit(sub_red_lst).top(time_filter = time_filter, limit = limit)
    
    update_data = []
    
    for d in data:
        d_data = {
            "ID": d.id,
            "Subreddit": str(d.subreddit),
            "Title": d.title,
            "Upvote": d.upvote_ratio,
            "Score": d.score,
            "Nums of Comment": d.num_comments
        }
        
        # Fetch post comments for sub-reddit posts
        
        posts = reddit.submission(d.id)
        posts.comments.replace_more(limit = None)
        
        for c in posts.comments.list():
            d_data["UserID"] = str(c.author)
            d_data["Comment"] = c.body
            update_data.append(d_data.copy())
            
    return pd.DataFrame(update_data)

# Part II: Get Data into DataFrame

In [13]:
data = data_fetch(sub_red_lst = "cscareerquestions+Jobs", 
                                             limit = 2000, 
                                             time_filter = "all")

TooManyRequests: received 429 HTTP response

# Part III: Export to CSV File

In [ ]:
data.to_csv("DATA.csv", header=True, index=False)